# Language in Space

## Session 10: static and dynamic maps

### Gerhard Jäger

January 13, 2022


In [ ]:
options(repr.plot.width=20, repr.plot.height=13)



## Constructing a Pacific-centered world map

In [ ]:
library(tidyverse)
library(sf)
library(spData)
library(tmap)
library(RColorBrewer)
library(leaflet)

By default, world maps are shown with the Greenwich meridian at the center.

In [ ]:
tm_shape(world) +
    tm_fill(col="darkgrey") +
    tm_graticules(alpha=0.2) +
    tm_layout(scale=3)

Different projections can be chosen with `st_transform` and a `proj4string`.

In [ ]:
world %>%
    st_transform("+proj=eqearth") %>%
    tm_shape() +
    tm_fill(col="darkgrey") +
    tm_graticules(alpha=0.2) +
    tm_layout(scale=3)

The central meridian of a projections can also modified via the `proj4string`, by altering the `lon_0` attribute:

`"+proj=eqearth lon_0=160"`

However, when we apply this, we get:

In [ ]:
world %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_fill(col="darkgrey") +
    tm_graticules(alpha=0.2) +
    tm_layout(scale=3) 

### Possible solution

If 160° East are at the center, 20° West are the outer boundary. The polygons transcending this border have to be cut into an eastern and a western half.

This means we have to divide the earth into **three** regions:
- 0°-160° East
- 160°-180° East
- 180°-0° West

First we have to make sure though that all geometries are interpreted as being on a plane (rather than on a sphere).

In [ ]:
sf_use_s2(FALSE)

In [ ]:
world %>%
    ggplot() +
    geom_sf() +
    geom_vline(xintercept=160, lwd=3, col='red') +
    geom_vline(xintercept=-20, lwd=3, col='blue') 

In [ ]:
(bb <- world %>% 
    st_bbox() %>% 
     c())


In [ ]:
xmn = bb[1]
xmx = bb[3]
ymn = bb[2]
ymx = bb[4]

In [ ]:
middle_hemisphere <- rbind(
  c(-20, ymn),
  c(160, ymn),
  c(160, ymx),
  c(-20, ymx),
  c(-20, ymn)
) %>% list() %>%
  st_polygon() %>%
  st_sfc()



In [ ]:
fareast <- rbind(
  c(160, ymn),
  c(xmx, ymn),
  c(xmx, ymx),
  c(160, ymx),
  c(160, ymn)
) %>% list() %>%
  st_polygon() %>%
  st_sfc()


In [ ]:
western_hemisphere <- rbind(
  c(-20, ymn),
  c(xmn, ymn),
  c(xmn, ymx),
  c(-20, ymx),
  c(-20, ymn)
) %>% list() %>%
  st_polygon() %>%
  st_sfc()


st_crs(middle_hemisphere) <- st_crs(western_hemisphere) <- st_crs(fareast) <- st_crs(world)

In [ ]:
world %>%
    tm_shape() +
    tm_fill() +
    tm_shape(middle_hemisphere) +
    tm_fill('red', alpha=0.2) +
    tm_shape(fareast) +
    tm_fill('green', alpha=0.2) +
    tm_shape(western_hemisphere) +
    tm_fill('blue', alpha=0.2)

Next we form the intersections of each region with all multipolygons in `world`.

In [ ]:
world.m <- world %>%
  st_intersection(middle_hemisphere)

world.w <- world %>%
  st_intersection(western_hemisphere)

world.e <- world %>%
  st_intersection(fareast)


In [ ]:
world.m %>%
    filter(continent=='Antarctica') %>%
    ggplot() + 
    geom_sf(fill='red') +
    xlim(-180, 180) +
    ylim(-90, 90)

In [ ]:
world.e %>%
    filter(continent=='Antarctica') %>%
    st_geometry()


In [ ]:
world.e %>%
    filter(continent=='Antarctica') %>%
    ggplot() + 
    geom_sf(fill='green') +
    xlim(-180, 180) +
    ylim(-100, 90)

In [ ]:
world.w %>%
    filter(continent=='Antarctica') %>%
    ggplot() + 
    geom_sf(fill='blue') +
    xlim(-180, 180) +
    ylim(-100, 90)

Adding CRSs.

In [ ]:
st_crs(world.e) <- st_crs(world.m) <- st_crs(world.w) <- 4326


In [ ]:
world.m %>%
    st_geometry() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() + tm_fill()

In [ ]:
world.w %>%
    st_geometry() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() + tm_fill()

In [ ]:
world.e %>%
    st_geometry() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() + tm_fill()

Evidently the multipolygons in the western hemisphere extend too far to the east for re-projection.

In [ ]:
sf_use_s2(FALSE)
lon.split = -20.01

western_hemisphere <- rbind(
  c(lon.split, ymn),
  c(xmn, ymn),
  c(xmn, ymx),
  c(lon.split, ymx),
  c(lon.split, ymn)
) %>% list() %>%
  st_polygon() %>%
  st_sfc()


st_crs(middle_hemisphere) <- st_crs(western_hemisphere) <- st_crs(fareast) <- st_crs(world)


world.w <- world %>%
  st_intersection(western_hemisphere)

In [ ]:
world.w %>%
    st_geometry() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() + tm_fill()

Next step: combining the three regions into one tibble.

In [ ]:
world.w <- world.w %>%
    mutate(region="west")
world.e <- world.e %>%
    mutate(region="fareast")
world.m <- world.m %>%
    mutate(region="middle")
    

In [ ]:
world.split <- rbind(
    world.w,
    world.e,
    world.m
)

In [ ]:
suppressMessages(
world.split %>%
    select(iso_a2) %>%
    group_by(iso_a2) %>%
    summarise() %>% 
    filter(iso_a2=="AQ") %>% 
    tm_shape() + tm_polygons() +
    tm_graticules() 
    )

In [ ]:
world.m %>%
    filter(continent=='Antarctica')

`group_by`/`summarise` let us combine the pieces back together

In [ ]:
suppressMessages(
    world.160e <- world.split %>%
    select(iso_a2) %>%
    group_by(iso_a2) %>%
    summarise()
    )

Now we have a tiny split at 20° West at all polygons transcending that line.

In [ ]:
world.160e %>%
    filter(iso_a2 == "AQ") %>%
    tm_shape() +
    tm_polygons() +
    tm_graticules()

This allows us to re-center the projection without distortions.

In [ ]:
world.160e %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_fill(col="darkgrey") +
    tm_graticules(lwd=0.1) +
    tm_layout(scale=3)

Let us save this `sf` object for later usage.

In [ ]:
world.160e %>% write_sf("data/world_160e.gpkg")

### Plotting the locations of WALS languages on this map



In [ ]:
walsLanguagesF = "data/languages.csv"

if (!file.exists(walsLanguagesF)) {
  download.file(
    "https://raw.githubusercontent.com/cldf-datasets/wals/master/cldf/languages.csv",
    dest = walsLanguagesF
    )
}

walsLanguages = read_csv(walsLanguagesF) %>%
  st_as_sf(coords=c("Longitude", "Latitude"))

st_crs(walsLanguages) <- 4326


In [ ]:
walsLanguages %>%
  st_geometry() %>%
  st_transform("+proj=eqearth lon_0=160") %>%
  tm_shape() +
  tm_symbols(size=.1, col='red', border.lwd=0) +
  tm_shape(world.160e) +
  tm_polygons(alpha=0)


removing country boundaries

In [ ]:
walsLanguages %>%
  st_geometry() %>%
  st_transform("+proj=eqearth lon_0=160") %>%
  tm_shape() +
  tm_symbols(size=.01, col='red', border.lwd=0) +
  tm_shape(
      world.160e %>%
      st_geometry() %>%
      st_union()
  ) +
  tm_polygons(alpha=0) +
  tm_layout(scale=3) +
  tm_graticules()

## Plotting language density per country

First step: joining the two tibbles.

In [ ]:
walsLanguages %>%
    select(ID) %>%
    st_join(world.160e) %>%
    st_drop_geometry() %>%
    group_by(iso_a2) %>%
    summarize(nLanguages = n()) %>%
    inner_join(world.160e) %>%
    st_as_sf() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_fill(col = "nLanguages", title="number of languages")
    

In [ ]:
worldWals <- walsLanguages %>%
    select(ID) %>%
    st_join(world.160e) %>%
    st_drop_geometry() %>%
    group_by(iso_a2) %>%
    summarize(nLanguages = n()) %>%
    inner_join(world.160e) %>%
    st_as_sf()
worldWals

To compute the language **density**, we need to pull the geographic size from `world` via `join`.

In [ ]:
world %>%
    st_drop_geometry() %>%
    select(iso_a2, name_long, area_km2) %>%
    inner_join(worldWals) %>%
    mutate(lDensity = 1000000 * nLanguages / area_km2) %>%
    drop_na() %>%
    ggplot() +
    geom_histogram(aes(x=lDensity)) +
    scale_x_continuous(trans="log10")


In [ ]:
world %>%
    st_drop_geometry() %>%
    select(iso_a2, name_long, area_km2) %>%
    inner_join(worldWals) %>%
    mutate(lDensity = 1000000 * nLanguages / (area_km2)) %>%
    mutate(logDensity = log(lDensity)) %>% 
    drop_na() %>%
    st_as_sf() %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_polygons(
        col = "lDensity", 
        midpoint=NA, 
        #breaks = c(-2, 0, 2, 4, 6, 8),
        #labels = c("0.01-1", "1-100", "100-10,000", "10,000-1,000,000", "1,000,000-100,000,000"),
        style="log10_pretty",
        title = "languages per 1M km_2",
        legend.hist=T
    ) +
   tm_style("col_blind") +
    tm_layout(legend.outside=T) 
    

## Plotting the distribution of typological features



In [ ]:

walsParametersF = "data/parameters.csv"


if (!file.exists(walsParametersF)) {
  download.file(
    "https://raw.githubusercontent.com/cldf-datasets/wals/master/cldf/parameters.csv",
    dest = walsParametersF
  )
}

(walsParameters = read_csv(walsParametersF))


In [ ]:

walsCodesF = "data/codes.csv"


if (!file.exists(walsCodesF)) {
  download.file(
    "https://raw.githubusercontent.com/cldf-datasets/wals/master/cldf/codes.csv",
    dest = walsCodesF
  )
}

(walsCodes = read_csv(walsCodesF))


In [ ]:

walsValuesF = "data/values.csv"


if (!file.exists(walsValuesF)) {
  download.file(
    "https://raw.githubusercontent.com/cldf-datasets/wals/master/cldf/values.csv",
    dest = walsValuesF
  )
}

(walsValues = read_csv(walsValuesF))


Let us focus on **Feature 81A: Order of Subject, Object and Verb**.

We want to replicate something like this: https://wals.info/feature/81A#2/18.0/152.9

In [ ]:
prm = "81A"

In [ ]:
(values.prm <- walsValues %>% 
    filter(Parameter_ID == prm) %>%
    select(Language_ID, Value))

In [ ]:
(values.codes.prm <- 
 walsCodes %>%
    filter(Parameter_ID == prm) %>%
    mutate(Value=Number) %>%
    select(Name, Value) %>%
    inner_join(values.prm) %>%
    select(!Value))

In [ ]:
(prmData <- walsLanguages %>%
    transmute(Language_ID = ID) %>%
    inner_join(values.codes.prm))

In [ ]:
options(warn=-1)
world.160e %>%
    st_transform("+proj=eqearth lon_0=160") %>%
    tm_shape() +
    tm_fill() +
    tm_graticules(lwd=0.3, n.x=10) + 
    tm_layout(scale=3) +
    tm_shape(prmData) +
    tm_dots(col="Name", size=0.1, border.lwd=0, title="word order") +
    tm_style("grey") +
    tm_layout(legend.outside=T) 

# Interactive maps

Static maps like those are good for publications, but most of the time we share data online. In this context we can add a lot of information via interactive features.

There are many frameworks for interactive map-maping out there. The package `leaflet` (actually a javascript package, with an R interface) is relatively mature and versatile.

In [ ]:
library(leaflet)

In [ ]:
m <- leaflet() %>%
  addTiles() %>%  # Add default OpenStreetMap map tiles
  addMarkers(lng=174.768, lat=-36.852, popup="The birthplace of R")
m  # Print the map

In [ ]:
m <- leaflet() %>%
  addTiles() %>%  # Add default OpenStreetMap map tiles
  addMarkers(lng=9.0619, lat=48.5266, popup="Seminar für Sprachwissenschaft")
m  # Print the map

There are several other map providers besides OpenStreetMap.

[Other map providers](https://leaflet-extras.github.io/leaflet-providers/preview/index.html)

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)

m <- leaflet() %>%
    addProviderTiles(providers$Esri.WorldImagery) %>%
  addMarkers(lng=9.0619, lat=48.5266, popup="Seminar für Sprachwissenschaft")
m

In [ ]:
m  <- leaflet() %>% setView(lng=9.0619, lat=48.5266, zoom=12)

In [ ]:
m %>% addTiles()

### Plotting the location of WALS languages colored according to language Family.

First we create a basemap, with a collection of background tiles to choose from

In [ ]:
basemap <- leaflet(height="1000", width="1400") %>%
  # add different provider tiles
  addProviderTiles(
    "OpenStreetMap",
    # give the layer a name
    group = "OpenStreetMap"
  ) %>%
  addProviderTiles(
    "Stamen.Toner",
    group = "Stamen.Toner"
  ) %>%
  addProviderTiles(
    "Stamen.Terrain",
    group = "Stamen.Terrain"
  ) %>%
  addProviderTiles(
    "Esri.WorldStreetMap",
    group = "Esri.WorldStreetMap"
  ) %>%
  addProviderTiles(
    "Wikimedia",
    group = "Wikimedia"
  ) %>%
  addProviderTiles(
    "CartoDB.Positron",
    group = "CartoDB.Positron"
  ) %>%
  addProviderTiles(
    "Esri.WorldImagery",
    group = "Esri.WorldImagery"
  ) %>%
# add a layers control
  addLayersControl(
    baseGroups = c(
      "OpenStreetMap", "Stamen.Toner",
      "Stamen.Terrain", "Esri.WorldStreetMap",
      "Wikimedia", "CartoDB.Positron", "Esri.WorldImagery"
    ),
    # position it on the topleft
    position = "topleft"
  )


In [ ]:
basemap

Next we need a function that maps Families to colors.

In [ ]:

factpal <- colorFactor(brewer.pal(n=12, name="Set3"), walsLanguages$Family)


Location markers are added with `addCircleMarkers`.

In [ ]:
interactiveWalsMap <- basemap %>%
    setView(lng=0, lat=30, zoom=2.5) %>%
    addCircleMarkers(
        data=walsLanguages,
        radius=1,
        stroke=T,
        weight=30,
        opacity=1,
        color = ~factpal(Family),
        clusterOptions = markerClusterOptions(),
        label = paste(
            "Name: ",
            walsLanguages$Name, "<br>",
            "Family:", walsLanguages$Family
        ) %>%
        lapply(htmltools::HTML),
        labelOptions = labelOptions(textsize = "20px")
    )

In [ ]:
interactiveWalsMap